# VotecounterTest
A script for setting up and executing performance tests on votecounters.

## Dependencies

In [1]:
import json
import time
from VoteCount import VoteCount
import VoteCounter
from tqdm import tqdm
import string

# to help generate formatted output representations
from IPython.core.display import display, HTML
import markdown2 as md

def html(markdown_string):
    display(HTML(md.markdown(markdown_string)))
    
# for simplifying text to just numbers
no_punctuation = str.maketrans(string.punctuation+string.ascii_letters, ' '*len(string.punctuation+string.ascii_letters))
    
# identify game archive and votecounter of interest
archive = '../data/archive.txt'
votecounter = VoteCounter

## Helper Function(s)

In [2]:
def _relevantGameInfo(game):
    link = game[:game.find('\n')]
    number = (link[link.find('&t=')+3:] if link.count('&')==1 
                      else link[link.find('&t=')+3:link.rfind('&')])
    title = game.split('\n')[1]
    title_number = [i for i in title.translate(no_punctuation).split() if i.isdigit()][0]
    moderators = game.split('\n')[2][len('Moderator: '):].split(', ')
    slots, players, correct = [], [], None
    for line in game[game.find('\nPlayers\n')+9:].split('\n'):
        line = line.split(', ')
        players += line[0].split(' replaced ')
        slots.append(line[0].split(' replaced ')) 
        if (line[2].lower().count('lynched') > 0 and
            line[2].lower().count('day 1') > 0):
            correct = slots[-1]
    return slots, players, correct, number, transitions[title_number], moderators

## VotecounterTest

In [146]:
#parameters
start_index = 0
end_index = 0

# open game archive, separate by game
with open(archive, encoding='utf-8') as f:
    games = f.read().split('\n\n\n')  

# open transitions archive, convert into dictionary of lists
with open('../data/transitions.tsv') as f:
    transitions = f.read()
transitions = transitions.split('\n')[3:]
transitions = {line.split('\t')[0]:line.strip().split('\t')[1:] for line in transitions if len(line.strip().split('\t')[1:]) > 0}

# process votes in each game's posts until a lynch found
# then store information about votecounter's performance
vote_results, vote_success, transition_results, transition_success, t0, total = {}, 0, {}, 0, time.time(), 0
end_index = end_index if end_index else len(games)  
for game_index, game in enumerate(games[start_index:end_index]):
    slots, players, correct, number, game_transitions, moderators = _relevantGameInfo(game)
    votecount = VoteCount(slots, meta={'correct': correct})
    votecounter = VoteCounter.VoteExtracter(players=players)

    # collect gameposts associated with game number
    with open('../data/posts/{}.jsonl'.format(number)) as f:
        gameposts =  [json.loads(l) for l in f]
        
    tgame, transition_start, transition_end, transition_match, transition_url = time.time(), None, None, False, None
    for post in gameposts:
        # consider no more votes if voters have made a choice already
        if not votecount.choice:

            # ignore posts not made by players
            if players.count(post['user']) == 0:
                continue

            # update votecount for each vote found by votecounter
            # stop considering votes in post if votecount.choice
            for voted in votecounter.fromPost(post):
                votecount.update(post['user'], voted, post['number'])
                if votecount.choice:
                    vote_success += votecount.choice == correct
                    break
        
        # keep scanning to find newest post by game mod after detectedhammer
        elif not transition_start:
            if moderators.count(post['user']) > 0:
                transition_start = int(post['number'])
                transition_url = post['pagelink']
        
        # keep scanning to find last successive post by mod after they end Day
        elif not transition_end:
            if moderators.count(post['user']) == 0:
                transition_end = int(post['number'])
                
                # track match between inferred and transcribed transition post#
                transition_match = int(game_transitions[0]) in list(range(transition_start, transition_end))
                transition_success += transition_match
        
        # finish if votecount.choice, transition_start, and transition_end all populated
        else:
            break
    
    transition_results[number] = [list(range(transition_start, transition_end)), transition_url] if transition_start and transition_end else "None"
    vote_results[number] = votecount
    total += 1
    if not transition_match:
        if '\nNotes: ' not in game.split('\n\n')[0]:
            print(game.split('\n\n')[0])
            print(game_index + start_index, number, vote_success, transition_success, total, votecount.choice == correct, transition_match, time.time()-tgame)
    
print(vote_success/float(end_index-start_index), time.time()-t0)

0.9362416107382551 361.64498233795166


## Analyze Results
Will have to revise this to adapt to new variable names/codeflow.

In [143]:
# parameters
game_index = 287
postnumber = 0
day = 1
day -= 1
postnumber = str(postnumber)

# open game archive, separate by game
with open(archive, encoding='utf-8') as f:
    games = f.read().split('\n\n\n')  

# generate votecounter w/ specified players
game = games[game_index]
slots, players, correct, number, game_transitions, moderators = _relevantGameInfo(game)
if not int(postnumber):
    postnumber = game_transitions[day]
    
html('# {}'.format(number))
print(games[game_index])
print()
print('Vote Choice:', vote_results[number].choice)
print('Vote Correct:', vote_results[number].meta['correct'])
votecount = VoteCount(slots, meta={'correct': correct})
votecounter = VoteCounter.VoteExtracter(players=players)

# collect gameposts associated with game number
with open('../data/posts/{}.jsonl'.format(number)) as f:
    gameposts = [json.loads(l) for l in f]
    
# find and display selected post along with votecounter output for it
html('# Post {}'.format(postnumber))
post = next(item for item in gameposts if item["number"] == postnumber)
print('Extracted Votes:', list(votecounter.fromPost(post)))
print()
print(post)
print()
display(HTML(post['content']))
    
# display extracted and true phase transitions
html('# Phase Transitions')
print('True Transitions:', game_transitions)
print(transition_results[number])
html('[{}]({})'.format(*transition_results[number]))
transition_start, transition_end, transition_match, final_vote = None, None, False, None
for post in gameposts[:int(postnumber)+10]:

    # consider no more votes if voters have made a choice already
    if not votecount.choice:

        # ignore posts not made by players
        if players.count(post['user']) == 0:
            continue

        # update votecount for each vote found by votecounter
        # stop considering votes in post if votecount.choice
        for voted in votecounter.fromPost(post):
            votecount.update(post['user'], voted, post['number'])
            if votecount.choice:
                print('Final Vote Post#', post['number'])
                break

    # keep scanning to find newest post by game mod after detectedhammer
    elif not transition_start:
        if moderators.count(post['user']) > 0:
            transition_start = int(post['number'])
            print('Transition Start:', transition_start)

    # keep scanning to find last successive post by mod after they end Day
    elif not transition_end:
        if moderators.count(post['user']) == 0:
            transition_end = int(post['number'])
            print('Transition End:', transition_start)

            # track match between inferred and transcribed transition post#
            transition_match = int(game_transitions[0]) in list(range(transition_start, transition_end))
            print('Transition Match:', transition_match)

    # finish if votecount.choice, transition_start, and transition_end all populated
    else:
        break
        
# display votecount up to indicated post number
html('# Current Votecount (Up to {})'.format(postnumber))
current_votecount = votecount.todict()
for each in current_votecount:
    if current_votecount[each]:
        print(each, '-', len(current_votecount[each]))
        for voter in current_votecount[each]:
            print(voter)
        print()

# display final votecount
html('# Final Votecount')
final_votecount = vote_results[number].todict()
for each in final_votecount:
    if final_votecount[each]:
        print(each, '-', len(final_votecount[each]))
        for voter in final_votecount[each]:
            print(voter)
        print()
        
# display final votelog
html('# Vote Log')
votelog = vote_results[number].votelog.copy()
for index, each in enumerate(votelog):
    each = each.split()
    each[-1] = '[{}]({})'.format(each[-1], next(item for item in gameposts if item["number"] == each[-1])['pagelink'])
    votelog[index] = ' '.join(each)
html('  \n'.join(reversed(votelog)))

https://forum.mafiascum.net/viewtopic.php?f=53&t=71908
Mini Normal 1914
Moderator: Fro99er, mhsmith0
Current Update: Mafia Win

Players
LaLight, Town Odd-Night Rolecop, lynched day 2
TwoFace, Vanilla Townie, killed night 4
Cooperative Sheep, Town 2-shot Rolestopper, killed night 1
Nachomamma8 replaced Mulch replaced Jordarrian, Vanilla Townie, lynched day 6
I Am Innocent, Vanilla Townie, endgamed
Vedith replaced Ingeel, Mafia Weak Doctor, killed night 3
BTD6_maker replaced noseacaballo, Town 2-Shot Roleblocker, lynched day 3
iDanyboy, Vanilla Townie, killed night 5
MarioManiac4, Town Motion Detector, lynched day 4
Dragnalus, Vanilla Townie, lynched day 1
Syndesis, Mafia Goon, lynched day 5
Alisae replaced TrueGent, Town Even-Night Rolecop, killed night 2
Dunkerdoodles replaced mush, Mafia Goon, survived

Vote Choice: ['Dragnalus']
Vote Correct: ['Dragnalus']


Extracted Votes: []

{'forum': '53', 'thread': '71908', 'pagelink': 'https://forum.mafiascum.net/viewtopic.php?f=53&t=71908&start=1050', 'number': '1068', 'timestamp': 'Sat Jun 03, 2017 11:04 am', 'user': 'Fro99er', 'content': '<div style="text-align: center"><span class="noboldsig"><span style="font-size: 300%; line-height: 116%;"><span style="font-family: \'Chiller\';"><span style="color:#000000 !important; background:#000000 !important; padding:0px 3px;" title="This text is hidden to prevent spoilers; to reveal, highlight it with your cursor."><span style="color: #FFBF00"><span class="nocolorsig"><img src="http://i.imgur.com/cANdMfU.png" alt="Image"><br><br>A lynch has been achieved...<br><br> <img src="./images/smilies/icon_surprised2.gif" alt=":o" title="Surprised">  <img src="./images/smilies/icon_surprised2.gif" alt=":o" title="Surprised">  <img src="./images/smilies/icon_surprised2.gif" alt=":o" title="Surprised">  <img src="./images/smilies/icon_surprised2.gif" alt=":o" title=

True Transitions: ['1068', '1247', '2135', '2372', '2398', '2341', '-']
[[1062], 'https://forum.mafiascum.net/viewtopic.php?f=53&t=71908&start=1050']


Final Vote Post# 929
Transition Start: 1062
Transition End: 1062
Transition Match: False


['I Am Innocent'] - 2
['Syndesis']
['BTD6_maker', 'noseacaballo']

['Vedith', 'Ingeel'] - 1
['Cooperative Sheep']

['Dragnalus'] - 7
['TwoFace']
['iDanyboy']
['Nachomamma8', 'Mulch', 'Jordarrian']
['LaLight']
['MarioManiac4']
['Dunkerdoodles', 'mush']
['Alisae', 'TrueGent']

['Alisae', 'TrueGent'] - 2
['Vedith', 'Ingeel']
['I Am Innocent']

Not Voting - 1
['Dragnalus']



['I Am Innocent'] - 2
['Syndesis']
['BTD6_maker', 'noseacaballo']

['Vedith', 'Ingeel'] - 1
['Cooperative Sheep']

['Dragnalus'] - 7
['TwoFace']
['iDanyboy']
['Nachomamma8', 'Mulch', 'Jordarrian']
['LaLight']
['MarioManiac4']
['Dunkerdoodles', 'mush']
['Alisae', 'TrueGent']

['Alisae', 'TrueGent'] - 2
['Vedith', 'Ingeel']
['I Am Innocent']

Not Voting - 1
['Dragnalus']



In [95]:
display(HTML(gameposts[1089]['content']))